In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
cap = cv2.VideoCapture(0)

while (cap.isOpened()):
    ret, frame = cap.read()
    cv2.imshow('OpenCv Cam', frame)
    if cv2.waitKey(10) & 0xFF == ord('q'): 
        break 
        
cap.release()
cv2.destroyAllWindows()